# Langgraph Agent

In [14]:
from langchain_openai import ChatOpenAI
from langchain_tavily import TavilySearch


In [38]:
from langgraph.graph import StateGraph, END, START
from langchain_core.tools import BaseTool
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_core.messages import ToolMessage


from typing_extensions import TypedDict
from typing import Annotated
import operator


class AgentState(TypedDict):
    messages: Annotated[list[BaseMessage], operator.add]

class LangGraphAgent:
    def __init__(self, model: ChatOpenAI, tools: list[BaseTool], system_message: str = ""):
        self.model = model.bind_tools(tools)
        self.tools = {t.name : t for t in tools}
        self.system_message = system_message
        
        graph = StateGraph(AgentState)

        graph.add_node("llm", self._call_llm)
        graph.add_node("action", self._call_action)

        graph.add_conditional_edges(
            "llm", self._exits_action, 
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")

        graph.set_entry_point("llm")
        self.graph = graph.compile()
    
    def _exits_action(self, state: AgentState) -> bool:
        last_message = state["messages"][-1]
        return last_message.tool_calls and len(last_message.tool_calls) > 0
    
    def _call_llm(self, state: AgentState) -> BaseMessage:
        messages = [SystemMessage(content=self.system_message)] + state["messages"]
        message = self.model.invoke(messages)
        return {"messages": [message]}
    
    def _call_action(self, state: AgentState) -> BaseMessage:
        results = []
        tool_calls = state["messages"][-1].tool_calls
        for tool_call in tool_calls:
            print(f"Calling Tool: {tool_call}")
            if tool_call["name"] in self.tools:
                tool = self.tools[tool_call["name"]]
                result = tool.invoke(tool_call["args"])
                print(f"Tool Result: {result}")
                results.append(ToolMessage(content=result, tool_call_id=tool_call["id"], name=tool_call["name"]))
        return {"messages": results}



In [45]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Load environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [46]:
model = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    model="qwen/qwen3-4b-2507",
    api_key=OPENAI_API_KEY,
    temperature=0.0,
)


tools = [
    TavilySearch(
        max_results=2, 
        tavily_api_key=TAVILY_API_KEY
    )
]

sys_prompt = """
You are a smart research assistant. Use the search engine to find the answer to the user's question.
You are allowed to make multiple calls (either together or in sequence).
Only look for information when you are sure of what you want.
If you see yourself repeating in a loop, stop and return the answer.
""".strip()

agent = LangGraphAgent(model=model, tools=tools, system_message=sys_prompt)

In [51]:
messages = [HumanMessage(content="What is the weather in SF?")]
result = agent.graph.invoke({"messages": messages})
result

Calling Tool: {'name': 'tavily_search', 'args': {'query': 'weather in SF', 'include_domains': ['weather.com', 'accuweather.com'], 'search_depth': 'basic', 'include_images': False}, 'id': '226418625', 'type': 'tool_call'}
Tool Result: {'query': 'weather in SF', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1756087230, 'localtime': '2025-08-24 19:00'}, 'current': {'last_updated_epoch': 1756087200, 'last_updated': '2025-08-24 19:00', 'temp_c': 16.1, 'temp_f': 61.0, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 11.0, 'wind_kph': 17.6, 'wind_degree': 257, 'wind_dir': 'WSW', 'pressure_mb': 1013.0, 'pressure_in': 2

{'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '226418625', 'function': {'arguments': '{"query":"weather in SF","include_domains":["weather.com","accuweather.com"],"search_depth":"basic","include_images":false}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 1884, 'total_tokens': 1935, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen/qwen3-4b-2507', 'system_fingerprint': 'qwen/qwen3-4b-2507', 'id': 'chatcmpl-i2bk5koge1d62s8vt9llf', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--67ad38f0-406c-4532-ad0b-e31bf11d8973-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'weather in SF', 'include_domains': ['weather.com', 'accuweather.com'], 'search_depth': 'basic', 'include_images': False}, 

In [52]:
result["messages"][-1].content

'The current weather in San Francisco (SF) is as follows:\n\n- **Temperature**: 16.1°C (61.0°F)\n- **Condition**: Overcast\n- **Wind**: 11.0 mph (17.6 kph) from the WSW direction\n- **Humidity**: 94%\n- **Cloud Coverage**: 100%\n- **Dew Point**: 13.4°C (56.0°F)\n- **Wind Chill**: 14.0°C (57.1°F)\n- **UV Index**: 0.2 (very low)\n\nIt is currently daytime, and there is no precipitation. The temperature feels similar to the actual temperature due to high humidity and overcast conditions. \n\nFor more details, you can refer to the official weather page.'

In [ ]:
# It doesn't remember the previous conversation.
# as there is no memory. The AgentState is only for one execution.
result =agent.graph.invoke({"messages": [HumanMessage(content="How about Dallas?")]})
result["messages"][-1].content

"I don't have enough context to provide a meaningful response about Dallas. Could you please clarify your question or provide more details? For example, are you asking about Dallas as a city, its economy, weather, tourism, or something else?"

In [ ]:

result =agent.graph.invoke({"messages": [HumanMessage(content="What is the weather in SF and LA?")]})
result["messages"][-1].content

Calling Tool: {'name': 'tavily_search', 'args': {'query': 'current weather in SF and LA', 'include_domains': ['weather.com', 'accuweather.com']}, 'id': '101337140', 'type': 'tool_call'}


"The current weather in San Francisco (SF) and Los Angeles (LA) is as follows:\n\n### **Los Angeles, CA**\n- **Temperature**: 92°F (33.4°C)\n- **Condition**: Sunny\n- **Wind**: 9.6 mph (SW direction)\n- **Humidity**: 26%\n- **Cloud Cover**: 0% (clear sky)\n- **Dew Point**: 64.6°F (18.1°C)\n- **UV Index**: 2.0\n- **Feels like**: 96°F (35.5°C) due to heat index\n\n### **San Francisco, CA**\nWhile the provided data does not include specific details for San Francisco, it does mention a separate weather forecast page for San Francisco, which includes current conditions, wind, air quality, and a 3-day forecast. For precise details on SF, you may refer to the [AccuWeather page](https://www.accuweather.com/en/us/san-francisco/94103/weather-forecast/347629) for updated and detailed information.\n\nLet me know if you'd like further details!"

In [ ]:
# the agent goes in loop if we ask for weather in SF and Dallas.
# so I updated the system prompt to stop the agent from going in loop.
result =agent.graph.invoke({"messages": [HumanMessage(content="What is the weather in SF and Dallas?")]})
result["messages"][-1].content

Calling Tool: {'name': 'tavily_search', 'args': {'query': 'current weather in SF and Dallas', 'include_domains': ['weather.com', 'accuweather.com']}, 'id': '829888646', 'type': 'tool_call'}
Tool Result: {'query': 'current weather in SF and Dallas', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco, Dallas', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Colonia San Francisco (San Francisco)', 'region': 'Zacatecas', 'country': 'Mexico', 'lat': 22.3036, 'lon': -101.8206, 'tz_id': 'America/Mexico_City', 'localtime_epoch': 1756082901, 'localtime': '2025-08-24 18:48'}, 'current': {'last_updated_epoch': 1756082700, 'last_updated': '2025-08-24 18:45', 'temp_c': 18.3, 'temp_f': 64.9, 'is_day': 1, 'condition': {'text': 'Light rain', 'icon': '//cdn.weatherapi.com/weather/64x64/day/296.png', 'code': 1183}, 'wind_mph': 10.5, 'wind_kph': 16.9, 'wind_degree': 75, 'wind_dir': 'ENE', 'pressure_mb': 1023.0, 'pressure_in'

'The provided search results do not contain accurate information about the current weather in San Francisco (SF), California, or Dallas, Texas. Instead, one result incorrectly lists a location as "Colonia San Francisco" in Zacatecas, Mexico, which is not relevant to the U.S. cities of interest.\n\nTo provide accurate and up-to-date weather information for San Francisco and Dallas, I recommend checking trusted weather websites directly, such as [AccuWeather](https://www.accuweather.com) or [Weather.com](https://www.weather.com). These sources provide real-time, location-specific weather details including temperature, precipitation, wind, and humidity for both cities. \n\nFor now, I cannot provide the correct weather details based on the retrieved results. Let me know if you\'d like further assistance!'

In [44]:
result =agent.graph.invoke({"messages": [HumanMessage(content="Which state won the superbowl in 2024 and what is the GDP of that state?")]})
result["messages"][-1].content

Calling Tool: {'name': 'tavily_search', 'args': {'query': 'which state won the superbowl in 2024', 'search_depth': 'advanced'}, 'id': '798705182', 'type': 'tool_call'}
Tool Result: {'query': 'which state won the superbowl in 2024', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.npr.org/2024/02/10/1230621176/super-bowl-58', 'title': 'Super Bowl 2024 Updates: The Kansas City Chiefs win the ... - NPR', 'content': "Kansas City Chiefs' tight end #87 Travis Kelce and Kansas City Chiefs' quarterback #15 Patrick Mahomes hug after winning Super Bowl LVIII against the San Francisco 49ers at Allegiant Stadium in Las Vegas, Nevada, February 11, 2024.\n\nThe Kansas City Chiefs win the 2024 Super Bowl, 25-22!\n\nThe Kansas City Chiefs have won their third Super Bowl title in five years, and are the first back-to-back NFL champions in almost 20 years. [...] NPR logo\n\nNPR Music\nNPR Music\n\n## Super Bowl 2024\n\n# Super Bowl 2024 updates: The commercials

"The Kansas City Chiefs won the 2024 Super Bowl (Super Bowl LVIII), defeating the San Francisco 49ers.\n\nHowever, the United States does not have a state-level GDP. GDP (Gross Domestic Product) is a measure of the total economic output of a country, and it is not typically attributed to individual states in the same way. While individual states may have their own economic data, such as state-level GDP, the official and most commonly referenced GDP figures are for the entire country.\n\nIf you're looking for the GDP of a specific U.S. state, please clarify, and I can provide further details. But for now, the Super Bowl winner is the Kansas City Chiefs, and there is no official GDP for a state in the U.S. context."